# Data Preprocessing

In [1]:
import os
print(f"Current working directory: {os.path.basename(os.getcwd())}")

# Change to root directory
os.chdir("../")
print(f"Current working directory (Changed): {os.path.basename(os.getcwd())}")

Current working directory: notebooks
Current working directory (Changed): Test-Store-Data-Analysis


In [10]:
#imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import json

# module setup
%matplotlib inline
pd.options.display.precision = 3
warnings.filterwarnings("ignore")

In [3]:
from src.constants import CONFIGS
from src.utils.basic_utils import read_yaml

In [4]:
configs = read_yaml(CONFIGS).data_preprocessing
dict(configs)

[2024-02-04 01:15:58 PM]:ProjectLogger INFO:basic_utils    39 - yaml file: conf\configs.yaml loaded successfully


{'scraped_data_path': 'data/external/scraped_products.csv',
 'processed_data_path': 'data/processed/products.csv'}

In [5]:
scraped_df = pd.read_csv(configs.scraped_data_path)
scraped_df.sample(5)

,title,price,in_stocks,sku,category,description,product_image_link,additional_info,product_link,scrape_ts
78,Helios Endurance Tank,£32.00,NaN,MT04,Tanks,"When training pushes your limits, you need gea...",https://gopher1.extrkt.com/wp-content/uploads/...,"{'Size': 'XS, S, M, L, XL', 'Color': 'Blue'}",https://gopher1.extrkt.com/?product=helios-end...,2024-02-03 22:52:57
124,Nora Practice Tank,£39.00,NaN,WT03,Erin Recommends|Clothing,"A closet go-to, the Nora Practice Tank can be ...",https://gopher1.extrkt.com/wp-content/uploads/...,"{'Size': 'XS, S, M, L, XL', 'Color': 'Orange, ...",https://gopher1.extrkt.com/?product=nora-pract...,2024-02-03 23:14:25
142,Riona Full Zip Jacket,£60.00,NaN,WJ05,Jackets,The Riona Basic Zip Jacket makes the perfect e...,https://gopher1.extrkt.com/wp-content/uploads/...,"{'Size': 'XS, S, M, L, XL', 'Color': 'Brown, G...",https://gopher1.extrkt.com/?product=riona-full...,2024-02-03 23:15:22
133,Portia Capri,£49.00,NaN,WP13,Erin Recommends|Clothing,From yoga at dawn to evenings curled up with a...,https://gopher1.extrkt.com/wp-content/uploads/...,"{'Size': '28, 29', 'Color': 'Blue, Green, Oran...",https://gopher1.extrkt.com/?product=portia-capri,2024-02-03 23:14:53
50,Diana Tights,£59.00,NaN,WP06,Erin Recommends|Clothing,Perfect for hot bikram session or cool-down st...,https://gopher1.extrkt.com/wp-content/uploads/...,"{'Size': '28, 29', 'Color': 'Black, Blue, Oran...",https://gopher1.extrkt.com/?product=diana-tights,2024-02-03 22:51:29


In [25]:
# if we don't split the "category" but keep it as it is

def clean_data(df: pd.DataFrame) -> pd.DataFrame:
    
    df = df[df["sku"] != "test-product"]
    df["price"] = df["price"].str.replace("£", "")
    df["in_stocks"] = df["in_stocks"].str.replace(" in stock", "")
    df["category"] = df["category"].str.replace("|", " ")
    
    # handled and removed in scraping (hot fix line: TO BE REMOVED)
    df['price'] = df['price'].str.replace("32.0024.00", "24.00")
    # ---- #
    df = df.astype({
        "price": "float32", 
        "in_stocks": "float32", 
        "scrape_ts": "datetime64[ns]"})
    
    info_list = []
    for _, row in df[["sku", "additional_info"]].iterrows():
        additional_info_dict = json.loads(row["additional_info"].replace("'", "\""))
        additional_info_dict["sku"] = row["sku"]
        info_list.append(additional_info_dict)
        
    info_df = pd.DataFrame(info_list)
    info_df.columns = map(lambda x: x.lower(), info_df.columns)
    
    # Column order required
    rearranged_col_list = [
        "sku", "title", "price_in_pounds", 
        "category", "description", "in_stocks",
        "product_image_link", "size", "color",
        "activity", "gender", "material", "pattern",
        "strap", "style", "product_link", "scrape_ts"
        ]
    
    final_df = (
        pd.merge(df, info_df, on="sku", how="inner")
        .drop(columns="additional_info")
        .rename(columns={"price": "price_in_pounds"})
        .reindex(columns = rearranged_col_list)
        .drop_duplicates(subset=["sku"])
        .reset_index(drop=True)
    )
    return final_df

In [26]:
main_df = clean_data(df=scraped_df)
main_df

,sku,title,price_in_pounds,category,description,in_stocks,product_image_link,size,color,activity,gender,material,pattern,strap,style,product_link,scrape_ts
0,MH09,Abominable Hoodie,69.0,Hoodies & Sweatshirts,It took CoolTech™ weather apparel know-how and...,NaN,https://gopher1.extrkt.com/wp-content/uploads/...,"XS, S, M, L, XL","Blue, Green, Red",NaN,NaN,NaN,NaN,NaN,NaN,https://gopher1.extrkt.com/?product=abominable...,2024-02-03 22:48:50
1,WJ08,Adrienne Trek Jacket,57.0,Erin Recommends Clothing,You’re ready for a cross-country jog or a coff...,NaN,https://gopher1.extrkt.com/wp-content/uploads/...,"XS, S, M, L, XL","Gray, Orange, Purple",NaN,NaN,NaN,NaN,NaN,NaN,https://gopher1.extrkt.com/?product=adrienne-t...,2024-02-03 22:48:54
2,WP07,Aeon Capri,48.0,Performance Fabrics Clothing,Reach for the stars and beyond in these Aeon C...,NaN,https://gopher1.extrkt.com/wp-content/uploads/...,"28, 29","Black, Blue, Orange",NaN,NaN,NaN,NaN,NaN,NaN,https://gopher1.extrkt.com/?product=aeon-capri,2024-02-03 22:48:57
3,MS01,Aero Daily Fitness Tee,24.0,Tees,Need an everyday action tee that helps keep yo...,NaN,https://gopher1.extrkt.com/wp-content/uploads/...,"XS, S, M, L, XL","Black, Brown, Yellow",NaN,NaN,NaN,NaN,NaN,NaN,https://gopher1.extrkt.com/?product=aero-daily...,2024-02-03 22:49:01
4,MP11,Aether Gym Pant,74.0,Erin Recommends Clothing,"The Aether Gym Pant is built for the studio, b...",NaN,https://gopher1.extrkt.com/wp-content/uploads/...,"32, 33, 34, 36","Blue, Brown, Green",NaN,NaN,NaN,NaN,NaN,NaN,https://gopher1.extrkt.com/?product=aether-gym...,2024-02-03 22:49:04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186,24-MB05,Wayfarer Messenger Bag,45.0,New Luma Yoga Collection,"Perfect for class, work or the gym, the Wayfar...",100.0,https://gopher1.extrkt.com/wp-content/uploads/...,NaN,NaN,"Gym, Overnight, Travel",NaN,"Nylon, Polyester",Solid,"Adjustable, Detachable, Double, Padded, Shoulder","Messenger, Laptop",https://gopher1.extrkt.com/?product=wayfarer-m...,2024-02-03 23:17:41
187,MP08,Zeppelin Yoga Pant,82.0,New Luma Yoga Collection Clothing,"Climb every mountain, or hold every pose, in t...",NaN,https://gopher1.extrkt.com/wp-content/uploads/...,"32, 33, 34, 36","Blue, Green, Red",NaN,NaN,NaN,NaN,NaN,NaN,https://gopher1.extrkt.com/?product=zeppelin-y...,2024-02-03 23:17:43
188,24-UG04,Zing Jump Rope,12.0,Fitness Equipment,One of the world’s simplest and most portable ...,100.0,https://gopher1.extrkt.com/wp-content/uploads/...,NaN,NaN,Gym,"Men, Women, Unisex","Leather, Plastic",NaN,NaN,NaN,https://gopher1.extrkt.com/?product=zing-jump-...,2024-02-03 23:17:47
189,WT02,Zoe Tank,29.0,Bras & Tanks,The Zoe Tank leads with eye-catching fitness-p...,NaN,https://gopher1.extrkt.com/wp-content/uploads/...,"XS, S, M, L, XL","Green, Orange, Yellow",NaN,NaN,NaN,NaN,NaN,NaN,https://gopher1.extrkt.com/?product=zoe-tank,2024-02-03 23:17:50


In [22]:
main_df.columns.to_list()

['sku',
 'title',
 'price_in_pounds',
 'category',
 'description',
 'in_stocks',
 'product_image_link',
 'size',
 'color',
 'activity',
 'gender',
 'material',
 'pattern',
 'strap',
 'style',
 'product_link',
 'scrape_ts']

['title',
 'price_in_pounds',
 'in_stocks',
 'sku',
 'category',
 'description',
 'product_image_link',
 'product_link',
 'scrape_ts',
 'size',
 'color',
 'activity',
 'gender',
 'material',
 'pattern',
 'strap',
 'style']

In [ ]:
["sku", "title", "price_in_pounds",
"category", "description", "in_stocks",
"product_image_link", "size", "color",
"activity", "gender", "material", "pattern",
"strap", "style", "product_link", "scrape_ts"
]


In [ ]:
# if we want to split the category

def clean_data_2(df: pd.DataFrame) -> pd.DataFrame:
    
    df = df[df["sku"] != "test-product"]
    df["price"] = df["price"].str.replace("£", "")
    df["in_stocks"] = df["in_stocks"].str.replace(" in stock", "")
    df = df.astype({
        "price": "float32", 
        "in_stocks": "float32", 
        "scrape_ts": "datetime64[ns]"})
    
    sub_df = (
        df["category"].str.split(pat='|', expand=True)
        .rename(columns={0: "category", 1: "category_group"})
        .fillna({"category_group": "Others"})
        )
    
    df = pd.concat([df.drop(columns="category"), sub_df], axis=1)
    
    info_list = []
    for _, row in df[["sku", "additional_info"]].iterrows():
        additional_info_dict = json.loads(row["additional_info"].replace("'", "\""))
        additional_info_dict["sku"] = row["sku"]
        info_list.append(additional_info_dict)
        
    info_df = pd.DataFrame(info_list)
    
    final_df = (
        pd.merge(df, info_df, on="sku", how="inner")
        .drop(columns="additional_info")
    )

In [22]:
sub_df = (
    scraped_df["category"].str.split(pat='|', expand=True)
    .rename(columns={0: "category", 1: "category_group"})
    .fillna({"category_group": "Others"})
)

df = pd.concat([scraped_df.drop(columns="category"), sub_df], axis=1)
df

,title,price,in_stocks,sku,description,product_image_link,additional_info,product_link,scrape_ts,category,category_group
0,Abominable Hoodie,£69.00,NaN,MH09,It took CoolTech™ weather apparel know-how and...,https://gopher1.extrkt.com/wp-content/uploads/...,"{'Size': 'XS, S, M, L, XL', 'Color': 'Blue, Gr...",https://gopher1.extrkt.com/?product=abominable...,2024-02-03 22:48:50,Hoodies & Sweatshirts,Others
1,Adrienne Trek Jacket,£57.00,NaN,WJ08,You’re ready for a cross-country jog or a coff...,https://gopher1.extrkt.com/wp-content/uploads/...,"{'Size': 'XS, S, M, L, XL', 'Color': 'Gray, Or...",https://gopher1.extrkt.com/?product=adrienne-t...,2024-02-03 22:48:54,Erin Recommends,Clothing
2,Aeon Capri,£48.00,NaN,WP07,Reach for the stars and beyond in these Aeon C...,https://gopher1.extrkt.com/wp-content/uploads/...,"{'Size': '28, 29', 'Color': 'Black, Blue, Oran...",https://gopher1.extrkt.com/?product=aeon-capri,2024-02-03 22:48:57,Performance Fabrics,Clothing
3,Aero Daily Fitness Tee,£24.00,NaN,MS01,Need an everyday action tee that helps keep yo...,https://gopher1.extrkt.com/wp-content/uploads/...,"{'Size': 'XS, S, M, L, XL', 'Color': 'Black, B...",https://gopher1.extrkt.com/?product=aero-daily...,2024-02-03 22:49:01,Tees,Others
4,Aether Gym Pant,£74.00,NaN,MP11,"The Aether Gym Pant is built for the studio, b...",https://gopher1.extrkt.com/wp-content/uploads/...,"{'Size': '32, 33, 34, 36', 'Color': 'Blue, Bro...",https://gopher1.extrkt.com/?product=aether-gym...,2024-02-03 22:49:04,Erin Recommends,Clothing
...,...,...,...,...,...,...,...,...,...,...,...
188,Wayfarer Messenger Bag,£45.00,100 in stock,24-MB05,"Perfect for class, work or the gym, the Wayfar...",https://gopher1.extrkt.com/wp-content/uploads/...,"{'Activity': 'Gym, Overnight, Travel', 'Patter...",https://gopher1.extrkt.com/?product=wayfarer-m...,2024-02-03 23:17:41,New Luma Yoga Collection,Others
189,Zeppelin Yoga Pant,£82.00,NaN,MP08,"Climb every mountain, or hold every pose, in t...",https://gopher1.extrkt.com/wp-content/uploads/...,"{'Size': '32, 33, 34, 36', 'Color': 'Blue, Gre...",https://gopher1.extrkt.com/?product=zeppelin-y...,2024-02-03 23:17:43,New Luma Yoga Collection,Clothing
190,Zing Jump Rope,£12.00,100 in stock,24-UG04,One of the world’s simplest and most portable ...,https://gopher1.extrkt.com/wp-content/uploads/...,"{'Activity': 'Gym', 'Gender': 'Men, Women, Uni...",https://gopher1.extrkt.com/?product=zing-jump-...,2024-02-03 23:17:47,Fitness Equipment,Others
191,Zoe Tank,£29.00,NaN,WT02,The Zoe Tank leads with eye-catching fitness-p...,https://gopher1.extrkt.com/wp-content/uploads/...,"{'Size': 'XS, S, M, L, XL', 'Color': 'Green, O...",https://gopher1.extrkt.com/?product=zoe-tank,2024-02-03 23:17:50,Bras & Tanks,Others


In [7]:
import pandas as pd

def clean_data(scraped_df):
    # Replace all instances of "£" with "" in column: 'price'
    scraped_df['price'] = scraped_df['price'].str.replace("£", "", case=False, regex=False)
    # Replace all instances of " in stock" with "" in column: 'in_stocks'
    scraped_df['in_stocks'] = scraped_df['in_stocks'].str.replace(" in stock", "", case=False, regex=False)
    # Replace all instances of "32.0024.00" with "24.00" in column: 'price'
    scraped_df['price'] = scraped_df['price'].str.replace("32.0024.00", "24.00", case=False, regex=False)
    # Change column type to float32 for column: 'price'
    scraped_df = scraped_df.astype({'price': 'float32'})
    # Change column type to float32 for column: 'in_stocks'
    scraped_df = scraped_df.astype({'in_stocks': 'float32'})
    # Change column type to datetime64[ns] for column: 'scrape_ts'
    scraped_df = scraped_df.astype({'scrape_ts': 'datetime64[ns]'})
    # Split text using string '|' in column: 'category'
    loc_0 = scraped_df.columns.get_loc('category')
    scraped_df_split = scraped_df['category'].str.split(pat='|', expand=True).add_prefix('category_')
    scraped_df = pd.concat([scraped_df.iloc[:, :loc_0], scraped_df_split, scraped_df.iloc[:, loc_0:]], axis=1)
    scraped_df = scraped_df.drop(columns=['category'])
    # Rename column 'category_0' to 'category'
    scraped_df = scraped_df.rename(columns={'category_0': 'category'})
    # Rename column 'category_1' to 'category_group'
    scraped_df = scraped_df.rename(columns={'category_1': 'category_group'})
    # Replace missing values with "Others" in column: 'category_group'
    scraped_df = scraped_df.fillna({'category_group': "Others"})
    return scraped_df

scraped_df_clean = clean_data(scraped_df.copy())
scraped_df_clean.head()

,title,price,in_stocks,sku,category,category_group,description,product_image_link,additional_info,product_link,scrape_ts
0,Abominable Hoodie,69.0,NaN,MH09,Hoodies & Sweatshirts,Others,It took CoolTech™ weather apparel know-how and...,https://gopher1.extrkt.com/wp-content/uploads/...,"{'Size': 'XS, S, M, L, XL', 'Color': 'Blue, Gr...",https://gopher1.extrkt.com/?product=abominable...,2024-02-03 22:48:50
1,Adrienne Trek Jacket,57.0,NaN,WJ08,Erin Recommends,Clothing,You’re ready for a cross-country jog or a coff...,https://gopher1.extrkt.com/wp-content/uploads/...,"{'Size': 'XS, S, M, L, XL', 'Color': 'Gray, Or...",https://gopher1.extrkt.com/?product=adrienne-t...,2024-02-03 22:48:54
2,Aeon Capri,48.0,NaN,WP07,Performance Fabrics,Clothing,Reach for the stars and beyond in these Aeon C...,https://gopher1.extrkt.com/wp-content/uploads/...,"{'Size': '28, 29', 'Color': 'Black, Blue, Oran...",https://gopher1.extrkt.com/?product=aeon-capri,2024-02-03 22:48:57
3,Aero Daily Fitness Tee,24.0,NaN,MS01,Tees,Others,Need an everyday action tee that helps keep yo...,https://gopher1.extrkt.com/wp-content/uploads/...,"{'Size': 'XS, S, M, L, XL', 'Color': 'Black, B...",https://gopher1.extrkt.com/?product=aero-daily...,2024-02-03 22:49:01
4,Aether Gym Pant,74.0,NaN,MP11,Erin Recommends,Clothing,"The Aether Gym Pant is built for the studio, b...",https://gopher1.extrkt.com/wp-content/uploads/...,"{'Size': '32, 33, 34, 36', 'Color': 'Blue, Bro...",https://gopher1.extrkt.com/?product=aether-gym...,2024-02-03 22:49:04


In [8]:
def clean_data(scraped_df_clean):
    # Drop duplicate rows in column: 'sku'
    scraped_df_clean = scraped_df_clean.drop_duplicates(subset=['sku'])
    # Select columns: 'sku', 'additional_info'
    scraped_df_clean = scraped_df_clean.loc[:, ['sku', 'additional_info']]
    return scraped_df_clean

scraped_df_clean_1 = clean_data(scraped_df_clean.copy())
scraped_df_clean_1.head()

,sku,additional_info
0,MH09,"{'Size': 'XS, S, M, L, XL', 'Color': 'Blue, Gr..."
1,WJ08,"{'Size': 'XS, S, M, L, XL', 'Color': 'Gray, Or..."
2,WP07,"{'Size': '28, 29', 'Color': 'Black, Blue, Oran..."
3,MS01,"{'Size': 'XS, S, M, L, XL', 'Color': 'Black, B..."
4,MP11,"{'Size': '32, 33, 34, 36', 'Color': 'Blue, Bro..."


In [9]:
import json

In [14]:
info_list = []
for idx, row in scraped_df_clean[["sku", "additional_info"]].iterrows():
    additional_info_dict = json.loads(row["additional_info"].replace("'", "\""))
    additional_info_dict["sku"] = row["sku"]
    info_list.append(additional_info_dict)


info_df = pd.DataFrame(info_list)
info_df.head()


,Size,Color,sku,Activity,Gender,Material,Pattern,Strap,Style
0,"XS, S, M, L, XL","Blue, Green, Red",MH09,NaN,NaN,NaN,NaN,NaN,NaN
1,"XS, S, M, L, XL","Gray, Orange, Purple",WJ08,NaN,NaN,NaN,NaN,NaN,NaN
2,"28, 29","Black, Blue, Orange",WP07,NaN,NaN,NaN,NaN,NaN,NaN
3,"XS, S, M, L, XL","Black, Brown, Yellow",MS01,NaN,NaN,NaN,NaN,NaN,NaN
4,"32, 33, 34, 36","Blue, Brown, Green",MP11,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192 entries, 0 to 191
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Size      158 non-null    object
 1   Color     155 non-null    object
 2   sku       192 non-null    object
 3   Activity  44 non-null     object
 4   Gender    30 non-null     object
 5   Material  44 non-null     object
 6   Pattern   14 non-null     object
 7   Strap     14 non-null     object
 8   Style     14 non-null     object
dtypes: object(9)
memory usage: 13.6+ KB


In [13]:
main_df = pd.merge(scraped_df_clean, info_df, on='sku', how='inner')
main_df

,title,price,in_stocks,sku,category,category_group,description,product_image_link,additional_info,product_link,scrape_ts,Size,Color,Activity,Gender,Material,Pattern,Strap,Style
0,Abominable Hoodie,69.0,NaN,MH09,Hoodies & Sweatshirts,Others,It took CoolTech™ weather apparel know-how and...,https://gopher1.extrkt.com/wp-content/uploads/...,"{'Size': 'XS, S, M, L, XL', 'Color': 'Blue, Gr...",https://gopher1.extrkt.com/?product=abominable...,2024-02-03 22:48:50,"XS, S, M, L, XL","Blue, Green, Red",NaN,NaN,NaN,NaN,NaN,NaN
1,Adrienne Trek Jacket,57.0,NaN,WJ08,Erin Recommends,Clothing,You’re ready for a cross-country jog or a coff...,https://gopher1.extrkt.com/wp-content/uploads/...,"{'Size': 'XS, S, M, L, XL', 'Color': 'Gray, Or...",https://gopher1.extrkt.com/?product=adrienne-t...,2024-02-03 22:48:54,"XS, S, M, L, XL","Gray, Orange, Purple",NaN,NaN,NaN,NaN,NaN,NaN
2,Aeon Capri,48.0,NaN,WP07,Performance Fabrics,Clothing,Reach for the stars and beyond in these Aeon C...,https://gopher1.extrkt.com/wp-content/uploads/...,"{'Size': '28, 29', 'Color': 'Black, Blue, Oran...",https://gopher1.extrkt.com/?product=aeon-capri,2024-02-03 22:48:57,"28, 29","Black, Blue, Orange",NaN,NaN,NaN,NaN,NaN,NaN
3,Aero Daily Fitness Tee,24.0,NaN,MS01,Tees,Others,Need an everyday action tee that helps keep yo...,https://gopher1.extrkt.com/wp-content/uploads/...,"{'Size': 'XS, S, M, L, XL', 'Color': 'Black, B...",https://gopher1.extrkt.com/?product=aero-daily...,2024-02-03 22:49:01,"XS, S, M, L, XL","Black, Brown, Yellow",NaN,NaN,NaN,NaN,NaN,NaN
4,Aether Gym Pant,74.0,NaN,MP11,Erin Recommends,Clothing,"The Aether Gym Pant is built for the studio, b...",https://gopher1.extrkt.com/wp-content/uploads/...,"{'Size': '32, 33, 34, 36', 'Color': 'Blue, Bro...",https://gopher1.extrkt.com/?product=aether-gym...,2024-02-03 22:49:04,"32, 33, 34, 36","Blue, Brown, Green",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,Wayfarer Messenger Bag,45.0,100.0,24-MB05,New Luma Yoga Collection,Others,"Perfect for class, work or the gym, the Wayfar...",https://gopher1.extrkt.com/wp-content/uploads/...,"{'Activity': 'Gym, Overnight, Travel', 'Patter...",https://gopher1.extrkt.com/?product=wayfarer-m...,2024-02-03 23:17:41,NaN,NaN,"Gym, Overnight, Travel",NaN,"Nylon, Polyester",Solid,"Adjustable, Detachable, Double, Padded, Shoulder","Messenger, Laptop"
189,Zeppelin Yoga Pant,82.0,NaN,MP08,New Luma Yoga Collection,Clothing,"Climb every mountain, or hold every pose, in t...",https://gopher1.extrkt.com/wp-content/uploads/...,"{'Size': '32, 33, 34, 36', 'Color': 'Blue, Gre...",https://gopher1.extrkt.com/?product=zeppelin-y...,2024-02-03 23:17:43,"32, 33, 34, 36","Blue, Green, Red",NaN,NaN,NaN,NaN,NaN,NaN
190,Zing Jump Rope,12.0,100.0,24-UG04,Fitness Equipment,Others,One of the world’s simplest and most portable ...,https://gopher1.extrkt.com/wp-content/uploads/...,"{'Activity': 'Gym', 'Gender': 'Men, Women, Uni...",https://gopher1.extrkt.com/?product=zing-jump-...,2024-02-03 23:17:47,NaN,NaN,Gym,"Men, Women, Unisex","Leather, Plastic",NaN,NaN,NaN
191,Zoe Tank,29.0,NaN,WT02,Bras & Tanks,Others,The Zoe Tank leads with eye-catching fitness-p...,https://gopher1.extrkt.com/wp-content/uploads/...,"{'Size': 'XS, S, M, L, XL', 'Color': 'Green, O...",https://gopher1.extrkt.com/?product=zoe-tank,2024-02-03 23:17:50,"XS, S, M, L, XL","Green, Orange, Yellow",NaN,NaN,NaN,NaN,NaN,NaN
